In [ ]:
# 57 - HOW TO GENERATE FEATURES FOR Machine Learning 

# https://github.com/bnsreenu/python_for_microscopists/blob/master/057-ML_06_02_what%20are%20features.py

In [1]:
import cv2
from skimage.filters.rank import entropy
from skimage.morphology import disk
from scipy import ndimage as nd
from skimage.filters import sobel
import pandas as pd

In [2]:
img = cv2.imread('./camera/2006251215008901.jpg') #RGB image -> convert to single color
img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
img = cv2.resize(img, (256, 256), 3)

In [3]:
# Create features vectors with Pandas df
# Convert into one single list or array 
# Unwrap my image into a 1D array

img2 = img.reshape(-1)
df = pd.DataFrame()
df['Original Pixel Values'] = img2

In [4]:
# 1. Entropy filter (lack of order), measure of disorder

entropy_img = entropy(img, disk(1))
entropy1 = entropy_img.reshape(-1)
df['Entropy'] = entropy1

In [5]:
# 2. Gaussian filter

gaussian_img = nd.gaussian_filter(img, sigma=3)
gaussian1 = gaussian_img.reshape(-1)
df['Gaussian'] = gaussian1

In [6]:
# 3. Sobel filter 

sobel_img = sobel(img)
sobel1 = sobel_img.reshape(-1)
df['Sobel'] = sobel1

In [ ]:
# 4. Check other tutorials for Gabor filter

In [7]:
df.head()

,Original Pixel Values,Entropy,Gaussian,Sobel
0,189,1.584963,149,0.0
1,195,2.000000,145,0.0
2,161,2.000000,138,0.0
3,153,2.000000,129,0.0
4,111,2.000000,121,0.0


In [ ]:
#Visualise Each: 

cv2.imshow('Entropy', entropy_img)
cv2.waitKey()
cv2.destroyAllWindows()

cv2.imshow('Gaussian', gaussian_img)
cv2.waitKey()
cv2.destroyAllWindows()

cv2.imshow('Sobel', sobel_img)
cv2.waitKey()
cv2.destroyAllWindows()

In [ ]:
###########

In [ ]:
# 51 - IMAGE SEGMENTATION USING K-MEANS

In [8]:
import numpy as np
import cv2

In [9]:
#Check openCV K-means Clustering documentation

im = cv2.imread('./camera/2006251815008927.jpg')
im = cv2.resize(im, (200,200),3)

im2 = im.reshape((-1,3))
im2 = np.float32(im2)

criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 10, 1.0)

#Clusters
k = 8

#times algorithm is repeated
attempts = 20 

ret,label,center=cv2.kmeans(im2,k,None,criteria,attempts,cv2.KMEANS_PP_CENTERS) #can use PP_centers or random_centers for how center is assigned 

# Now convert back into uint8 (unsigned integers), and make original image
center = np.uint8(center)
# Flatten labels
res = center[label.flatten()]

In [10]:
#Reshape them
res2 = res.reshape((im.shape))

In [11]:
cv2.imwrite('segmented.jpg', res2)

True

In [6]:
cv2.imshow('res2',res2)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
###############

In [ ]:
# 52 - IMAGE SEGMENTATION USING GAUSSIAN MIXTURE MODEL (GMM)

In [ ]:
# 2 approaches into different umbrellas:

# Unsupervised Learning
# Figure out where features are

# Supervised Learning 
# Supply a labeled image where you paint the pixels and label them
# Save model and use to train future models

# GMM is another example of Clustering technique falling into image/data processing
#Clusters overlap then GMM is more suitable because K-means has hard assignments

In [17]:
import numpy as np
import cv2

from sklearn.mixture import GaussianMixture as GMM

In [18]:
img = cv2.imread('./camera/2006251815008927.jpg')
img = cv2.resize(im, (200,200),3)
img2 = img.reshape((-1,3))

In [ ]:
# Step 1: Define instance, model
# Step 2: Fit it to data you have
# Step 3: Predict using data

In [19]:
#Invoke my GMM
gmm_model = GMM(n_components=2, covariance_type='tied').fit(img2)
#type is also spherical or diagonal not only tied, depends on type of data. Check documentation

#Generate labels
gmm_labels = gmm_model.predict(img2)

In [21]:
original_shape = img.shape
segmented = gmm_labels.reshape(original_shape[0], original_shape[1])
cv2.imwrite('segmented_im.jpg', segmented)

True

In [ ]:
## 66 - IMAGE SEGMENTATION - TRADITIONAL ML

# Code adapted from: https://github.com/bnsreenu/python_for_microscopists/blob/master/062-066-ML_06_04_TRAIN_ML_segmentation_All_filters_RForest.py
# STEP 1: FEATURE EXTRACTION

In [ ]:
# Code to train SUPPORT VECTOR MACHINE for image segmentation
import numpy as np
import cv2
import pandas as pd

img = cv2.imread('./camera/2006251300008904.jpg')
img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

In [ ]:
#Add features to dataframe needed for our ML algorithm
#Pixel value tells a lot about what each region is

#Feature #1 is our original pixel values to the data frame added
img2 = img.reshape(-1)
df = pd.DataFrame()
df['Original Image'] = img2
#print(df.head)

In [ ]:
#Add other features 

# First set: Gabor Features (like gaussian or any kind of edge
# detection features). Create a gabor filter bank to generate reponses onto the original image. 


# Generate Gabor features: this combination gives 32 different filters
num = 1
kernels = []
for theta in range(2):
    theta = theta / 4. * np.pi
    for sigma in (1,3):
        for lamda in np.arange(0, np.pi, np.pi / 4):
            for gamma in (0.05,0.5):
                
                gabor_label = 'Gabor' + str(num)
                ksize = 5 #or 9
                kernel = cv2.getGaborKernel((ksize, ksize), sigma, theta, lamda, gamma, 0, ktype=cv2.CV_32F)
                kernels.append(kernel)
                #Now fikter the image and add values to a new column
                fimg = cv2.filter2D(img2, cv2.CV_8UC3, kernel)
                filtered_img = fimg.reshape(-1)
                df[gabor_label] = filtered_img #labels columns as Gabor1, Gabor2 
                print(gabor_label, ': theta=', theta, ':sigma=', sigma, ': lamda=', lamda, ': gamma=', gamma)
                num += 1 #Increment for gabor column label

print(df.head())                

In [ ]:
df.head(2)

In [ ]:
# Add other feature extractors: 

# Canny Edge: Edge detection 
edges = cv2.Canny(img, 100, 200)
edges1 = edges.reshape(-1)
df['Canny Edge'] = edges1

df.head()

In [ ]:
from skimage.filters import roberts, sobel, scharr, prewitt

#ROBERTS EDGE
edge_roberts = roberts(img)
edge_roberts1 = edge_roberts.reshape(-1)
df['Roberts'] = edge_roberts1

#SOBEL
edge_sobel = sobel(img)
edge_sobel1 = edge_sobel.reshape(-1)
df['Sobel'] = edge_sobel1

#SCHARR
edge_scharr = scharr(img)
edge_scharr1 = edge_scharr.reshape(-1)
df['Scharr'] = edge_scharr1

#PREWITT
edge_prewitt = prewitt(img)
edge_prewitt1 = edge_prewitt.reshape(-1)
df['Prewitt'] = edge_prewitt1

#GAUSSIAN with sigma=3
from scipy import ndimage as nd
gaussian_img = nd.gaussian_filter(img, sigma=3)
gaussian_img1 = gaussian_img.reshape(-1)
df['Gaussian s3'] = gaussian_img1

#GAUSSIAN with sigma=7
gaussian_img2 = nd.gaussian_filter(img, sigma=7)
gaussian_img3 = gaussian_img2.reshape(-1)
df['Gaussian s7'] = gaussian_img3

#MEDIAN with sigma=3
median_img = nd.median_filter(img, size=3)
median_img1 = median_img.reshape(-1)
df['Median s3'] = median_img1

#VARIANCE with size=3
# slow part, we may not need that
# variance_img = nd.generic_filter(img, np.var, size=3)
# variance_img1 = variance_img.reshape(-1)
# df['Variance s3'] = variance_img1  #Add column to original dataframe

df.head(3)

In [ ]:
# Add another column that corresponds to the feature image
# We need to tell what ground truth is (training)
labeled_img = cv2.imread('./2105011200000000.png')
labeled_img = cv2.cvtColor(labeled_img, cv2.COLOR_BGR2GRAY)
labeled_img1 = labeled_img.reshape(-1)
df['Labels'] = labeled_img1

df.head(3)

In [ ]:
# STEP 2: TRAINING A MODEL (USING RANDOM FOREST CLASSIFIER RFC)

# Define dependent variable Y 
Y = df['Labels'].values

# Define independent variable X
X = df.drop(labels = ['Labels'], axis=1) #keep everything except Labels column

In [ ]:
# Split data into test and train
#from sklearn.model_selection import train_test_split
#X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.4, random_state=20)

#Import ML algorithm and train model

# here we use random forest as our classifier:
# RandomForestRegressor works too but predicts a number value of data 
#from sklearn.ensemble import RandomForestClassifier
#model = RandomForestClassifier(n_estimators=10, random_state=42)

In [ ]:
############# USE SVM Method ##############

from sklearn.svm import LinearSVC
model = LinearSVC(max_iter=100) #try 1000 too (accuracy increases)

model.fit(X_train, Y_train)

##############

In [ ]:
prediction_train = model.predict(X_train)
#use predict.proba for probability
prediction_test = model.predict(X_test)

In [ ]:
#Metrics for accuracy
from sklearn import metrics

#compare
#Y_test is our True 
print('Accuracy =', metrics.accuracy_score(Y_test, prediction_test))

In [ ]:
# STEP 4: SAVING TRAINED MODEL
#Use this model to segment other images

import pickle
filename = 'ourfirst_model'
pickle.dump(model, open(filename, 'wb')) # wb= write in binary format

In [ ]:
#Use the model to train: import it 
load_model = pickle.load(open(filename, 'rb')) #rb = read binary mode
result = load_model.predict(X) #Predict all pixels in X

segmented = result.reshape((img.shape))

#Visualise
from matplotlib import pyplot as plt
plt.imshow(segmented, cmap='jet')
plt.imsave('segmented_img.jpg', segmented, cmap='jet')

#If I'm happy with this segmentation as a working image, let's apply
# that to all the images and move to production mode. 

#Place all images in folder train_images in the Segmented folder 
#To do so, create a copy of this notebook file up until here

In [ ]:
# STEP 5: SEGMENTING MULTIPLE IMAGES USING A SAVED MODEL

#Feature extraction 
# Code to train random forest for image segmentation
import numpy as np
import cv2
import pandas as pd

#########
# STEP 5 - Adding code here 

#Function:
def feature_extraction(img): #img that we'll supply later
    df = pd.DataFrame() #Create empty dataframe
    
    #Add original pixel values to df as feature #1
    img2 = img.reshape(-1)
    df['Original Image'] = img2
    #print(df.head)

#Add other features 
# First set: Gabor Features
# Generate Gabor features
    num = 1
    kernels = []
    for theta in range(2):
        theta = theta / 4. * np.pi
        for sigma in (1,3):
            for lamda in np.arange(0, np.pi, np.pi / 4):
                for gamma in (0.05,0.5):
                
                    gabor_label = 'Gabor' + str(num)
                    ksize = 5 #or 9
                    kernel = cv2.getGaborKernel((ksize, ksize), sigma, theta, lamda, gamma, 0, ktype=cv2.CV_32F)
                    kernels.append(kernel)
                #Now filter the image and add values to a new column
                    fimg = cv2.filter2D(img2, cv2.CV_8UC3, kernel)
                    filtered_img = fimg.reshape(-1)
                    df[gabor_label] = filtered_img #labels columns as Gabor1, Gabor2 
                    print(gabor_label, ': theta=', theta, ':sigma=', sigma, ': lamda=', lamda, ': gamma=', gamma)
                    num += 1 #Increment for gabor column label

#print(df.head())  
# Add other feature extractors: 
    # Canny Edge: Edge detection 
    edges = cv2.Canny(img, 100, 200)
    edges1 = edges.reshape(-1)
    df['Canny Edge'] = edges1

    from skimage.filters import roberts, sobel, scharr, prewitt

    #ROBERTS EDGE
    edge_roberts = roberts(img)
    edge_roberts1 = edge_roberts.reshape(-1)
    df['Roberts'] = edge_roberts1

    #SOBEL
    edge_sobel = sobel(img)
    edge_sobel1 = edge_sobel.reshape(-1)
    df['Sobel'] = edge_sobel1

    #SCHARR
    edge_scharr = scharr(img)
    edge_scharr1 = edge_scharr.reshape(-1)
    df['Scharr'] = edge_scharr1

    #PREWITT
    edge_prewitt = prewitt(img)
    edge_prewitt1 = edge_prewitt.reshape(-1)
    df['Prewitt'] = edge_prewitt1

    #GAUSSIAN with sigma=3
    from scipy import ndimage as nd
    gaussian_img = nd.gaussian_filter(img, sigma=3)
    gaussian_img1 = gaussian_img.reshape(-1)
    df['Gaussian s3'] = gaussian_img1

    #GAUSSIAN with sigma=7
    gaussian_img2 = nd.gaussian_filter(img, sigma=7)
    gaussian_img3 = gaussian_img2.reshape(-1)
    df['Gaussian s7'] = gaussian_img3

    #MEDIAN with sigma=3
    median_img = nd.median_filter(img, size=3)
    median_img1 = median_img.reshape(-1)
    df['Median s3'] = median_img1

#VARIANCE with size=3
# slow part, we may not need that
# variance_img = nd.generic_filter(img, np.var, size=3)
# variance_img1 = variance_img.reshape(-1)
# df['Variance s3'] = variance_img1  #Add column to original dataframe
    
    return df

########
# Everything from previous steps is useless because we are using
# unknown data

In [ ]:
import glob
import pickle
from matplotlib import pyplot as plt

#Import the pickled file
filename = './2006251300008904.jpg'
load_model = pickle.load(open(filename, 'rb')) #reading binary mode

In [ ]:
#Step through the folder containing images that need to be segmented

#Go through each file - multiple files
#Same as steps before except we use the for loop to step through files
path = 'images/Train_images/*.jpg'
for file in glob.glob(path):
    img1 = cv2.imread(file)
    img = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
    
    #Do feature extraction and call the function which returns a df
    X = feature_extraction(img)
    result = load_model.predict(X) #the pickled model and predict
    segmented = result.reshape((img.shape)) #reshape to my original img shape
    name = file.split("e_")
    plt.imsave('images/Segmented/' + name[1], segmented, cmap='jet')

In [ ]:
# Image segmentation approach where we generated a bunch of features
# from our input images and then trained a ML algo model (in this case RF)
# saved the model and used it to segment a whole bunch of images
# in a folder.

#Leveraging the greatness of Gabor filter using texture to 
# perform segmentation

# for limited amount of training images, traditional ML 
# beats DL. If you have more than 100 images for training, then 
# use DL and U-net approach.

# Source: https://github.com/bnsreenu/python_for_microscopists

In [ ]:
#########

#What are Gabor filters? 

import matplotlib.pyplot as plt
plt.imshow(kernel)

kernel_resized = cv2.resize(kernel, (400,400)) 

cv2.imshow('Original Image', img)
cv2.imshow('Filtered Image', fimg)
cv2.imshow('Kernel', kernel_resized)
cv2.waitKey()
cv2.destroyAllWindows()

#df.to_csv('Gabor.csv')


In [ ]:
## 67 - Feature based image segmentation using traditional Machine Learning

# Code adapted from: https://github.com/bnsreenu/python_for_microscopists/blob/master/067-ML_06_05_PREDICT_ML_segmentation_All_filters_RForest.py

# Multi-training images 
# Good idea to label masks and images the same way 

import numpy as np
import cv2
import pandas as pd

import glob
import pickle
from matplotlib import pyplot as plt

In [ ]:
#Applying trained model to segment multiple files. 

filename = "ourfirst_model"
loaded_model = pickle.load(open(filename, 'rb'))

path = "./images/Train_images/*.jpg"
for file in glob.glob(path):
    print(file)     #just stop here to see all file names printed
    img1= cv2.imread(file)
    img1 = cv2.resize(img1, (128,128))
    img = cv2.cvtColor(img1,cv2.COLOR_BGR2GRAY)

#Call the feature extraction function.
    X = feature_extraction(img)
    result = loaded_model.predict(X)
    segmented = result.reshape((img.shape))
    
    name = file.split("e_")
    plt.imsave('images/Segmented/'+ name[1], segmented, cmap ='jet')

#Above, we are splitting the file path into 2 -> creates a list with 2 entries
#Then we are taking the second half of name to save segmented images with that name

In [ ]:
def feature_extraction(img):
    df = pd.DataFrame()


#All features generated must match the way features are generated for TRAINING.
#Feature1 is our original image pixels
    img2 = img.reshape(-1)
    df['Original Image'] = img2

#Generate Gabor features
    num = 1
    kernels = []
    for theta in range(2):
        theta = theta / 4. * np.pi
        for sigma in (1, 3):
            for lamda in np.arange(0, np.pi, np.pi / 4):
                for gamma in (0.05, 0.5):
#               print(theta, sigma, , lamda, frequency)
                
                    gabor_label = 'Gabor' + str(num)
#                    print(gabor_label)
                    ksize=9
                    kernel = cv2.getGaborKernel((ksize, ksize), sigma, theta, lamda, gamma, 0, ktype=cv2.CV_32F)    
                    kernels.append(kernel)
                    #Now filter image and add values to new column
                    fimg = cv2.filter2D(img2, cv2.CV_8UC3, kernel)
                    filtered_img = fimg.reshape(-1)
                    df[gabor_label] = filtered_img  #Modify this to add new column for each gabor
                    num += 1
                    
########################################

#Generate OTHER FEATURES and add them to the data frame
#Feature 3 is canny edge
    edges = cv2.Canny(img, 100,200)   #Image, min and max values
    edges1 = edges.reshape(-1)
    df['Canny Edge'] = edges1 #Add column to original dataframe

    from skimage.filters import roberts, sobel, scharr, prewitt

#Feature 4 is Roberts edge
    edge_roberts = roberts(img)
    edge_roberts1 = edge_roberts.reshape(-1)
    df['Roberts'] = edge_roberts1

#Feature 5 is Sobel
    edge_sobel = sobel(img)
    edge_sobel1 = edge_sobel.reshape(-1)
    df['Sobel'] = edge_sobel1

#Feature 6 is Scharr
    edge_scharr = scharr(img)
    edge_scharr1 = edge_scharr.reshape(-1)
    df['Scharr'] = edge_scharr1

    #Feature 7 is Prewitt
    edge_prewitt = prewitt(img)
    edge_prewitt1 = edge_prewitt.reshape(-1)
    df['Prewitt'] = edge_prewitt1

    #Feature 8 is Gaussian with sigma=3
    from scipy import ndimage as nd
    gaussian_img = nd.gaussian_filter(img, sigma=3)
    gaussian_img1 = gaussian_img.reshape(-1)
    df['Gaussian s3'] = gaussian_img1

    #Feature 9 is Gaussian with sigma=7
    gaussian_img2 = nd.gaussian_filter(img, sigma=7)
    gaussian_img3 = gaussian_img2.reshape(-1)
    df['Gaussian s7'] = gaussian_img3

    #Feature 10 is Median with sigma=3
    median_img = nd.median_filter(img, size=3)
    median_img1 = median_img.reshape(-1)
    df['Median s3'] = median_img1

    #Feature 11 is Variance with size=3
    variance_img = nd.generic_filter(img, np.var, size=3)
    variance_img1 = variance_img.reshape(-1)
    df['Variance s3'] = variance_img1  #Add column to original dataframe


    return df


#########################################################

In [ ]:
## 26 - Denoising and edge detection using OpenCV 

# Code adapted from: https://github.com/bnsreenu/python_for_microscopists/blob/master/026-image_processing_in_openCV_intro1-preprocessing.py

In [ ]:
import cv2
import numpy as np
from matplotlib import pyplot as plt

In [ ]:
# Denoising

img = cv2.imread('./camera/2006251145008899.jpg')
img = cv2.resize(img, (512,512))
kernel = np.ones((5,5),np.float32)/25
filt_2D = cv2.filter2D(img,-1,kernel)    #Convolution using the kernel we provide
blur = cv2.blur(img,(5,5))   #Convolution with a normalized filter. Same as above for this example.
blur_gaussian = cv2.GaussianBlur(img,(5,5),0)  #Gaussian kernel is used. 
median_blur = median = cv2.medianBlur(img,5)  #Using kernel size 5. Better on edges compared to gaussian.
bilateral_blur = cv2.bilateralFilter(img,9,75,75)  #Good for noise removal but retain edge sharpness. 

In [ ]:
cv2.imshow("Original", img)
cv2.imshow("2D filtered", filt_2D)
cv2.imshow("Blur", blur)
cv2.imshow("Gaussian Blur", blur_gaussian)
cv2.imshow("Median Blur", median_blur)
cv2.imshow("Bilateral", bilateral_blur)
cv2.waitKey(0)          
cv2.destroyAllWindows() 

In [ ]:
#Edge detection:
    
import cv2
import numpy as np

img = cv2.imread("./camera/2006251145008899.jpg", 1)
img = cv2.resize(img, (512,512))
edges = cv2.Canny(img,100,200)   #Image, min and max values

cv2.imshow("Original Image", img)
cv2.imshow("Canny", edges)

cv2.waitKey(0)          
cv2.destroyAllWindows() 

In [ ]:
# Many ways to open images in Python.
# PIL
# matplotlib
# skimage
# openCV
# other libraries to open propriatery images like czi, OME-TIFF

# 1. Using PIL

from PIL import Image 
import numpy as np   #Use numpy to convert images to arrays

In [ ]:
# Read image 
img = Image.open("./camera/2006251145008899.jpg") #Not a numpy array
print(type(img))

In [ ]:
# Output Images 
img.show() 

In [ ]:
# prints format of image 
print(img.format)

In [ ]:
# prints mode of image 
print(img.mode) 

In [ ]:
#PIL is not by default numpy array but can convert PIL image to numpy array. 
img1 = np.asarray(img)
print(type(img1))

In [ ]:
# 2. Using Matplotlib
import matplotlib.image as mpimg 
import matplotlib.pyplot as plt

In [ ]:
img = mpimg.imread("./camera/2006251145008899.jpg")  #this is a numpy array
print(type(img))
print(img)

print(img.shape)

plt.imshow(img)
plt.colorbar()   #Puts a color bar next to the image. 

In [ ]:
# 3. Using Scikit Image

# Includes algorithms for segmentation, feature detection etc. 
# Great for Random forest or SVM
from skimage import io, img_as_float, img_as_ubyte
import numpy as np
import matplotlib.pyplot as plt 

In [ ]:
image = img_as_float(io.imread("./2006251145008899.jpg"))

#image2 = io.imread("images/test_image.jpg").astype(np.float)
#avoid using astype as it violates assumptions about dtype range.
#for example float should range from 0 to 1 (or -1 to 1) but if you use 
#astype to convert to float, the values do not lie between 0 and 1. 
print(image.shape)
plt.imshow(img)

In [ ]:
print(image)

In [ ]:
image8byte = img_as_ubyte(image)
print(image8byte)

In [ ]:
# 4. using OpenCV

# You can import images in color, grey scale or unchanged using 
# individual commands 
# cv2.IMREAD_COLOR : Loads a color image. Any transparency of image 
# will be neglected. It is the default flag.
# cv2.IMREAD_GRAYSCALE : Loads image in grayscale mode
# cv2.IMREAD_UNCHANGED : Loads image as such including alpha channel
# Instead of these three flags, you can simply pass integers 1, 0 
# or -1 respectively.

import cv2

grey_img = cv2.imread("./camera/2006251145008899.jpg", 0)
grey_img = cv2.resize(grey_img, (256,256))
color_img = cv2.imread("./camera/2006251145008899.jpg", 1)
color_img = cv2.resize(color_img, (256,256))

#images opened using cv2 are numpy arrays
print(type(grey_img)) 
print(type(color_img)) 

In [ ]:
# Use the function cv2.imshow() to display an image in a window. 
# First argument is the window name which is a string. second argument is our image. 
cv2.imshow("pic", grey_img)
cv2.imshow("color pic", color_img)

# Maintain output window until 
# user presses a key or 1000 ms (1s)
cv2.waitKey(0)          

#destroys all windows created
cv2.destroyAllWindows() 

In [ ]:
#OpenCV imread, imwrite and imshow all work with the BGR order, not RGB
#but there is no need to change the order when you read an image with 
#cv2.imread and then want to show it with cv2.imshow
#if you use matplotlib, it uses RGB. 

import matplotlib.pyplot as plt
plt.imshow(color_img)

In [ ]:
  #OpenCV represents RGB images as multi-dimensional NumPy arrays, but as BGR.

#we can convert the images from BGR to RGB
#plt.imshow(cv2.cvtColor(color_img, cv2.COLOR_BGR2RGB))

#We can also change color spaces from RGB to HSV..
plt.imshow(cv2.cvtColor(color_img, cv2.COLOR_BGR2HSV))

In [ ]:
# 5. Using Ome-tiff

import apeer_ometiff_library
from apeer_ometiff_library import io  #Use apeer.com free platform for image processing in the cloud

#OME-TIFF has tiff and metada (as XML) embedded
#Image is a 5D array.

(pic2, omexml) = io.read_ometiff("./camera/2006251145008899.tif")  #Unwrap image and embedded xml metadata
print (pic2.shape)   #to verify the shape of the array
print(pic2)

print(omexml)

In [ ]:
# 6. CZI files 

import czifile

img = czifile.imread('images/test_image.czi')
print(img.shape)


import czifile
from skimage import io

img = czifile.imread('images/Osteosarcoma_01.czi')
print(img.shape)
img1=img[0, 0, :, :, :, 0]
print(img1.shape)
img2=img1[2,:,:]
io.imshow(img2)

In [ ]:
# 7. Reading multiple images from a folder

# Glob module finds all the path names
#matching a specified pattern according to the rules used by the Unix shell
#The glob.glob returns the list of files with their full path 

import cv2
import glob

In [ ]:
# Select the path

# /*.jpg 
path = "images/test_images/aeroplane/*.*"
for file in glob.glob(path):
    print(file)     #just stop here to see all file names printed
    a= cv2.imread(file)  #now, we can read each file since we have the full path
    print(a)  #print numpy arrays for each file

In [ ]:
# Let us look at each file
    cv2.imshow('Original Image', a)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [ ]:
# Process each image - change color from BGR to RGB.
    c = cv2.cvtColor(a, cv2.COLOR_BGR2RGB)
    cv2.imshow('Color image', c)
#wait for 1 second
    k = cv2.waitKey(0)
#destroy the window
    cv2.destroyAllWindows()

In [ ]:
## 29 - Features, detectors, keypoints

# Code adapted from: https://github.com/bnsreenu/python_for_microscopists/blob/master/029-keypoint%20detectors%20and%20descriptors%20in%20opencv.py

In [ ]:
# 1. Harris corner 
import cv2
import numpy as np

In [ ]:
img = cv2.imread('./camera/2006251145008899.jpg')
img = cv2.resize(img, (256,256))
gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

gray = np.float32(gray)  #Harris works on float32 images. 

In [ ]:
#Input parameters
# image, block size (size of neighborhood considered), ksize (aperture parameter for Sobel), k
harris = cv2.cornerHarris(gray,2,3,0.04)  

# Threshold for an optimal value, it may vary depending on the image.
img[harris>0.01*harris.max()]=[255,0,0]    # replace these pixels with blue

cv2.imshow('Harris Corners',img)
cv2.waitKey(0)

In [ ]:
# 2. Shi-Tomasi Corner Detector & Good Features to Track
# In opencv it is called goodfeaturestotrack
import cv2
import numpy as np

img = cv2.imread('./camera/2006251145008899.jpg')
img = cv2.resize(img, (256,256))
gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

#input image, #points, quality level (0-1), min euclidean dist. between detected points
corners = cv2.goodFeaturesToTrack(gray,50,0.05,10)
corners = np.int0(corners)   #np.int0 is int64

for i in corners:
    x,y = i.ravel()   # Ravel Returns a contiguous flattened array.
#    print(x,y)
    cv2.circle(img,(x,y),3,255,-1)  #Draws circle (Img, center, radius, color, etc.)

cv2.imshow('Corners',img)
cv2.waitKey(0)

In [ ]:
# 3. FAST 
# Features from Accelerated Segment Test
# High speed corner detector
# FAST is only keypoint detector. Cannot get any descriptors. 

#SIFT and SURF - do not work in opencv 3
#SIFT stands for scale invariant feature transform

import cv2
import matplotlib.pyplot as plt

img = cv2.imread('./camera/2006251145008899.jpg', 0)
img = cv2.resize(img, (512,512))

# Initiate FAST object with default values
detector = cv2.FastFeatureDetector_create(50)   #Detects 50 points

kp = detector.detect(img, None)

img2 = cv2.drawKeypoints(img, kp, None, flags=0)

plt.imshow(img2)
cv2.imshow('Corners',img2)
cv2.waitKey(0)

In [ ]:
# 4. ORB
# Oriented FAST and Rotated BRIEF
# An efficient alternative to SIFT or SURF
# ORB is basically a fusion of FAST keypoint detector and BRIEF descriptor

#BRIEF (Binary Robust Independent Elementary Features)
#One important point is that BRIEF is a feature descriptor, 
#it doesn’t provide any method to find the features.
# Not going to show the example as BRIEF also not working in opencv 3
import numpy as np
import cv2

img = cv2.imread('./camera/2006251145008899.jpg', 0)
img = cv2.resize(img, (512,512))

orb = cv2.ORB_create(100)
kp, des = orb.detectAndCompute(img, None)

# draw only keypoints location,not size and orientation
#img2 = cv2.drawKeypoints(img, kp, None, flags=None)
# Now, let us draw with rich key points, reflecting descriptors. 
# Descriptors here show both the scale and the orientation of the keypoint.
img2 = cv2.drawKeypoints(img, kp, None, flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
cv2.imshow("With keypoints", img2)
cv2.waitKey(0)

plt.imshow(img2)